In [3]:
!pip show opencv-python

Name: opencv-python
Version: 4.8.1.78
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Software\Anaconda\Lib\site-packages
Requires: numpy, numpy, numpy, numpy, numpy
Required-by: ultralytics


In [1]:
import numpy as np
import cv2
import cv2.aruco as aruco




## Marker corners detected in real time

Code referenced from:
https://docs.opencv.org/4.8.0/d5/dae/tutorial_aruco_detection.html

In [2]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Change grayscale
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)  # Use 5x5 dictionary to find markers
    parameters = aruco.DetectorParameters()  
    detector = aruco.ArucoDetector(aruco_dict, parameters)

    corners, ids, rejected_img_points = detector.detectMarkers(gray)
       
    # If aruco markers are detected:
    if np.all(ids is not None):  
        for i in range(0, len(ids)):  
                 aruco.drawDetectedMarkers(frame, corners) 
                    
                    
    cv2.imshow('frame', frame)
        
    key = cv2.waitKey(3) & 0xFF
    if key == ord('q'):  
        break
    
cap.release()
cv2.destroyAllWindows()



## Getting marker coordinates

Thymio:    index 0
Obstacles: indicies 1-6
Goal:      index 7

In [3]:
def current_photo():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cv2.imwrite('images/current.jpg', frame)
    cap.release()

In [4]:
current_photo()

The following functions below retrieve the first photo, and utilize it to locate the coordinates of the markers.
Aruco uses X right, Y far, Z up, if the marker is lying flat on a table. Origin is in the center of the marker. The top left corner of the marker is considered the "first" corner.

In [36]:
def get_Thymio():
    img_path = 'images/current.jpg'
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Change grayscale
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)  # Use 5x5 dictionary to find markers
    parameters = aruco.DetectorParameters()  
    detector = aruco.ArucoDetector(aruco_dict, parameters)

    corners, ids, rejected_img_points = detector.detectMarkers(gray)
    
    # If aruco markers are detected:
    # if np.all(ids is not None):
      #  output_image = gray.copy()
        #aruco.drawDetectedMarkers(output_image,corners) 
                    
    # Get thymio at idx 0  
    idx = np.where(ids == 0)[0][0]
    coords = corners[idx]
    
    return coords, idx
    


In [37]:
corners, ids = get_Thymio()


In [38]:
print(corners)
print(ids)

[[[170. 249.]
  [214. 249.]
  [210. 287.]
  [166. 289.]]]
6


In [61]:
def get_obst():
    img_path = 'images/current.jpg'
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Change grayscale
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)  # Use 5x5 dictionary to find markers
    parameters = aruco.DetectorParameters()  
    detector = aruco.ArucoDetector(aruco_dict, parameters)

    corners, ids, rejected_img_points = detector.detectMarkers(gray)
    
    # If aruco markers are detected:
    # if np.all(ids is not None):
      #  output_image = gray.copy()
        #aruco.drawDetectedMarkers(output_image,corners) 
                    
    # Get obstacles at idx that is not Thymio or goal 
    idx = np.where((ids != 0) & (ids != 7))[0]


    coords = [corners[i].tolist() for i in idx]

        
    return coords, idx

In [62]:
corners, ids = get_obst()
print(corners)
print(ids)

[[[[161.0, 318.0], [207.0, 318.0], [201.0, 362.0], [154.0, 362.0]]], [[[464.0, 238.0], [509.0, 238.0], [514.0, 282.0], [468.0, 281.0]]], [[[191.0, 397.0], [241.0, 397.0], [239.0, 449.0], [189.0, 448.0]]], [[[470.0, 314.0], [517.0, 314.0], [522.0, 362.0], [474.0, 362.0]]], [[[315.0, 313.0], [362.0, 314.0], [362.0, 361.0], [314.0, 360.0]]], [[[315.0, 244.0], [359.0, 241.0], [361.0, 281.0], [314.0, 282.0]]]]
[0 1 3 4 5 7]


In [39]:
def get_goal():
    img_path = 'images/current.jpg'
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Change grayscale
    aruco_dict = aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)  # Use 5x5 dictionary to find markers
    parameters = aruco.DetectorParameters()  
    detector = aruco.ArucoDetector(aruco_dict, parameters)

    corners, ids, rejected_img_points = detector.detectMarkers(gray)
    
    # If aruco markers are detected:
    # if np.all(ids is not None):
      #  output_image = gray.copy()
        #aruco.drawDetectedMarkers(output_image,corners) 
                    
    # Get goal at idx 7  
    idx = np.where(ids == 7)[0][0]
    coords = corners[idx]
    
    return coords, idx

In [40]:
corners, ids = get_goal()
print(corners)
print(ids)

[[[437. 397.]
  [485. 397.]
  [489. 448.]
  [439. 449.]]]
2


## Atempt pose estimation

### 1. First need to calibrate camera
See calibrateCamera()
https://docs.opencv.org/4.8.0/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d